<a href="https://colab.research.google.com/github/uhdang/mlim_group_3_final_assignment/blob/main/project/train_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:

- Downsample training set (reduce method to delete irrelevant rows)
- maybe PCA()

# Google Colab Runtime
Run if using Google Colab

In [1]:
# # # import pandas as pd
# # # 
# # import numpy as np# 
# # 
# import o# s# 
# 
# from google.colab import dri# v# e# 
# 
# import # s# y# s#
# drive.mount('/content/dri# v# e# '# )
# sys.path.append('/content/drive/My Drive/My Dri# v# e# '# )
#
# from dataloader import Dat# a# l# o# ader
# # L# o# a# d#  Data
# path = "/content/drive/MyDrive/My Drive/"

# Local Runtime
Run if using notebook locally

In [1]:
import pandas as pd
import os
import optuna.integration.lightgbm as lightgb
import pickle
import sklearn

from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn'

from dataloader import Dataloader
from dataloader import create_combined_dict


# Load Data
path = os.getcwd() + "/../data/"

# Load Data

In [3]:
# load data which creates baskets and coupons
data = Dataloader(path)

In [4]:
# Create Categories before CV as they don't change and consume time otherwise
data.create_category_table(10000)

In [5]:
%%capture
weeks = [86, 87, 88, 89]
shopper_list = list(range(2000))
shopper_chunks = [shopper_list[i:i + 100] for i in range(0, len(shopper_list), 100)]

cv_dict = {
    'X_train': list(),
    'y_train': list(),
    'X_test': list(),
    'y_test': list()
}

In [6]:
for i, week in enumerate(weeks):
  print(f"week: {week}")

  X_train_list = list()
  y_train_list = list()
  X_test_list = list()
  y_test_list = list()

  for idx, shopper in enumerate(tqdm(shopper_chunks)):
      # print(f"shopper_chunk index: {idx}")

      # train-test-split
      data.train_test_split(week, shopper)

      # data add categories
      data.add_categories()

      # create features
      data.create_feature_dict()

      # combine everything
      if i == 0:
          X_train, y_train, X_test, y_test = data.make_featured_data()
          X_train_list.append(X_train)
          y_train_list.append(y_train)
          del X_train, y_train
      else:
          _, _, X_test, y_test = data.make_featured_data()

      X_test_list.append(X_test)
      y_test_list.append(y_test)
      del X_test, y_test

  cv_dict = create_combined_dict(X_train_list, y_train_list, X_test_list, y_test_list, cv_dict)

100%|██████████| 20/20 [49:15<00:00, 147.78s/it]


week: 86
week: 87
week: 88
week: 89


In [8]:
with open('cv_dict.pickle', 'wb') as handle:
    pickle.dump(cv_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [37]:
with open('dataloader.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

feature_fraction, val_score: inf:   0%|          | 0/7 [40:06<?, ?it/s]


# Hyperparameter Tuning

In [10]:
# %%capture
# !pip install optuna

In [2]:
# Load Data From Pickle

with open(os.getcwd() + "/../data/pickle/cv_dict_01.pickle", 'rb') as f:
  cv_dict =  pickle.load(f)

In [7]:
sklearn.__version__

'0.24.1'

In [3]:
def downsample(df, y):
  df_target_coupon = df.loc[(y==1) | (df['coupon']=='Yes')]
  y_target_coupon = y.loc[(y==1) | (df['coupon']=='Yes')]
  df_down = df.loc[(y==0) & (df['coupon']=='No')]
  y_down = y.loc[(y==0) & (df["coupon"]=="No")]
  df_down, y_down = resample(
      df_down,
      y_down,
      replace=False,
      n_samples=df_target_coupon.shape[0],
      stratify=df_down['shopper']
  )
  df_all = pd.concat([df_target_coupon, df_down], ignore_index=True)
  y_all = pd.concat([y_target_coupon, y_down], ignore_index=True)

  return df_all, y_all

In [4]:
dtrain = lightgb.Dataset(cv_dict['X_train'][0], label=cv_dict['y_train'][0])
X_test0, y_test0 = downsample(cv_dict['X_test'][0], cv_dict['y_test'][0])
dval0 = lightgb.Dataset(X_test0, label=y_test0)
X_test1, y_test1 = downsample(cv_dict['X_test'][1], cv_dict['y_test'][1])
dval1 = lightgb.Dataset(X_test1, label=y_test1)
X_test2, y_test2 = downsample(cv_dict['X_test'][2], cv_dict['y_test'][2])
dval2 = lightgb.Dataset(X_test2, label=y_test2)
X_test3, y_test3 = downsample(cv_dict['X_test'][3], cv_dict['y_test'][3])
dval3 = lightgb.Dataset(X_test3, label=y_test3)

In [6]:

params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

lgb_clf = lightgb.train(
    params,
    dtrain,
    categorical_feature=['shopper', 'product', 'category', 'coupon', 'coupon_in_same_category'],
    valid_sets=[dval0, dval1, dval2],
    verbose_eval=100,
    early_stopping_rounds=100
)

[I 2021-03-20 22:54:44,548] A new study created in memory with name: no-name-8b1fe5f3-3cb6-4547-b686-50556ed40fc4

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]C:\Users\uh_da\anaconda3\envs\hu-mlim\lib\site-packages\lightgbm\basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['category', 'coupon', 'coupon_in_same_category', 'product', 'shopper']
  warnings.warn('categorical_feature in Dataset is overridden.\n'
[W 2021-03-20 22:55:40,965] Trial 0 failed because of the following error: ValueError('Series.dtypes must be int, float or bool')
Traceback (most recent call last):
  File "C:\Users\uh_da\anaconda3\envs\hu-mlim\lib\site-packages\optuna\_optimize.py", line 217, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\uh_da\anaconda3\envs\hu-mlim\lib\site-packages\optuna\integration\_lightgbm_tuner\optimize.py", line 249, in __call__
    booster = lgb.train(self.lgbm_params, train_set, **kwargs)
  F

ValueError: Series.dtypes must be int, float or bool

In [ ]:
prediction = lgb_clf.predict_proba(cv_dict['X_test'][3], num_iteration=lgb_clf.best_iteration)[:,1]

In [ ]:
AUC = roc_auc_score(
    y_true = cv_dict['y_test'][3],
    y_score = prediction
)

In [ ]:
best_params = lgb_clf.params
print("Best params:", best_params)
print("  Accuracy = {}".format(AUC))
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

# Prediction and Revenue Optimization

# Playground

In [ ]:
# train-test-split
data.train_test_split(89, list(range(50)))

# data add categories 
data.add_categories()
data.create_feature_dict()
X_train = data._make_full_table(data.baskets_train, data.coupons_train)
X_test = data._make_full_table(data.baskets_test, data.coupons_test)

X_train = data._merge_features(X_train, data.feature_dict)
X_test = data._merge_features(X_test, data.feature_dict)

In [6]:
X_train.head()

,week,shopper,product,price,target,category,discount,coupon,original_price,ratio_product_count,ratio_category_count,reordered_product,reordered_category,coupon_in_same_category,ratio_of_reordered_products,ratio_of_reordered_categories,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper
0,0,0,0,688.0,0.0,12,0.0,No,688,0.000000,0.089888,0.0,1.0,Yes,0.722222,0.960000,587.203947,8.539326,54,25
1,0,1,0,688.0,0.0,12,0.0,No,688,0.067416,0.314607,1.0,1.0,Yes,0.830986,1.000000,584.065350,7.393258,71,24
2,0,2,0,688.0,0.0,12,0.0,No,688,0.000000,0.595506,0.0,1.0,No,0.700000,0.954545,571.918919,5.820225,60,22
3,0,3,0,688.0,0.0,12,0.0,No,688,0.000000,0.415730,0.0,1.0,No,0.714286,1.000000,561.846361,8.337079,91,24
4,0,4,0,688.0,0.0,12,0.0,No,688,0.000000,0.089888,0.0,1.0,No,0.764706,1.000000,583.750455,6.168539,34,20
